# Deep Learning Project

In [ ]:
%cd /content/
!rm -rf DLProject/
!git clone https://github.com/ManuelaCorte/DLProject.git
!git checkout -d decoder origin/decoder
%cd /content/DLProject/

In [ ]:
%pip install ftfy regex tqdm ultralytics wandb albumentations torchviz
%pip install git+https://github.com/openai/CLIP.git

In [ ]:
import sys
import os
import gdown

sys.path.append("DLProject/src/")

In [ ]:
# Download dataset and save under data/raw/ only if not already downloaded
url = "https://drive.google.com/uc?id=1xijq32XfEm6FPhUb7RsZYWHc2UuwVkiq"
if not os.path.exists("data/raw/refcocog.tar.gz"):
    print("Downloading dataset...")
    gdown.download(url=url, output="data/raw/", quiet=False, resume=True)
if not os.path.exists("data/raw/refcocog/"):
    print("Extracting dataset...")
    !tar -xf data/raw/refcocog.tar.gz -C data/raw

The preprocessed samples can be downloaded from Google Drive by executing the following cell. Otherwise, the preprocessing wil be done saving the file only temporarly in the Colab environment. 

In [ ]:
# Download preprocessed dataset
url = "https://drive.google.com/drive/folders/1jaJV40dneOckZn7WHMQyd2jBh7A8534N"
gdown.download_folder(url=url, output="data/", quiet=False)

In [ ]:
%cd src

In [ ]:
from vgproject.train import main

main()